In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import os.path
import time
import sys
import random
import tensorflow as tf
import numpy as np
import importlib
import argparse
import facenet
import lfw
import h5py
import tensorflow.contrib.slim as slim
from tensorflow.python.ops import data_flow_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops

In [2]:
network = importlib.import_module('models.inception_resnet_v1')

In [3]:
logs_base_dir = '/mnt/data/facenet/logs/facenet'
models_base_dir = '/mnt/data/facenet/models/facenet'
data_dir = '/mnt/data/Face_dataset/lfw_crop'
lfw_dir = '/mnt/data/Face_dataset/lfw_crop'
lfw_pairs = '/mnt/data/facenet/data/pairs.txt'
lfw_file_ext= 'png'
embedding_size = 512
nrof_preprocess_threads = 1
image_size_arg = 160
image_size = (160,160)
epoch_size = 1000
batch_size = 90
seed = 666
weight_decay = 5e-4
keep_probability = 0.8
filter_filename = ''
filter_percentile = 100.0
filter_min_nrof_images_per_class = 0
validation_set_split_ratio = 0.05
min_nrof_val_images_per_class = 0

In [4]:
subdir = datetime.strftime(datetime.now(), '%Y%m%d-%H%M%S')
log_dir = os.path.join(os.path.expanduser(logs_base_dir), subdir)
if not os.path.isdir(log_dir):  # Create the log directory if it doesn't exist
    os.makedirs(log_dir)
model_dir = os.path.join(os.path.expanduser(models_base_dir), subdir)
if not os.path.isdir(model_dir):  # Create the model directory if it doesn't exist
    os.makedirs(model_dir)

In [7]:
facenet.write_arguments_to_file(args, os.path.join(log_dir, 'arguments.txt'))

NameError: name 'args' is not defined

In [5]:
src_path,_ = os.path.split(os.path.realpath(__file__))
facenet.store_revision_info(src_path, log_dir, ' '.join(sys.argv))

NameError: name '__file__' is not defined

In [6]:
np.random.seed(seed=seed)
random.seed(seed)
dataset = facenet.get_dataset(data_dir)
if filter_filename:
    dataset = filter_dataset(dataset, os.path.expanduser(filter_filename), 
        filter_percentile, filter_min_nrof_images_per_class)

if validation_set_split_ratio>0.0:
    train_set, val_set = facenet.split_dataset(dataset, validation_set_split_ratio, min_nrof_val_images_per_class, 'SPLIT_IMAGES')
else:
    train_set, val_set = dataset, []

In [7]:
len(dataset)
# len(val_set)

5749

In [7]:
print('Model directory: %s' % model_dir)
print('Log directory: %s' % log_dir)
pretrained_model = None

Model directory: /mnt/data/facenet/models/facenet/20180831-153257
Log directory: /mnt/data/facenet/logs/facenet/20180831-153257


In [15]:
if lfw_dir:
    print('LFW directory: %s' % lfw_dir)
    # Read the file containing the pairs used for testing
    pairs = lfw.read_pairs(os.path.expanduser(lfw_pairs))
    # Get the paths for the corresponding images
    lfw_paths, actual_issame = lfw.get_paths(os.path.expanduser(lfw_dir), pairs)

LFW directory: /mnt/data/Face_dataset/lfw_crop


In [54]:
len(actual_issame)

6000

In [21]:
graph = tf.get_default_graph()

In [22]:
tf.set_random_seed(seed)
global_step = tf.Variable(0, trainable=False)

In [75]:
# Get a list of image paths and their labels
image_list, label_list = facenet.get_image_paths_and_labels(train_set)
assert len(image_list)>0, 'The dataset should not be empty'
val_image_list, val_label_list = facenet.get_image_paths_and_labels(val_set)

In [63]:
len(image_list)

13233

In [28]:
# Create a queue that produces indices into the image_list and label_list 
labels = ops.convert_to_tensor(label_list, dtype=tf.int32)
range_size = array_ops.shape(labels)[0]
index_queue = tf.train.range_input_producer(range_size, num_epochs=None,
                     shuffle=True, seed=None, capacity=32)

In [29]:
index_queue

In [30]:
index_dequeue_op = index_queue.dequeue_many(batch_size*epoch_size, 'index_dequeue')

learning_rate_placeholder = tf.placeholder(tf.float32, name='learning_rate')
batch_size_placeholder = tf.placeholder(tf.int32, name='batch_size')
phase_train_placeholder = tf.placeholder(tf.bool, name='phase_train')
image_paths_placeholder = tf.placeholder(tf.string, shape=(None,1), name='image_paths')
labels_placeholder = tf.placeholder(tf.int32, shape=(None,1), name='labels')
control_placeholder = tf.placeholder(tf.int32, shape=(None,1), name='control')

In [31]:
input_queue = data_flow_ops.FIFOQueue(capacity=2000000,
                                    dtypes=[tf.string, tf.int32, tf.int32],
                                    shapes=[(1,), (1,), (1,)],
                                    shared_name=None, name=None)
enqueue_op = input_queue.enqueue_many([image_paths_placeholder, labels_placeholder, control_placeholder], 
                                      name='enqueue_op')

In [51]:
enqueue_op

<tf.Operation 'enqueue_op_1' type=QueueEnqueueManyV2>

In [42]:
image_batch, label_batch = facenet.create_input_pipeline(input_queue, image_size, nrof_preprocess_threads, batch_size_placeholder)

In [49]:
label_batch

<tf.Tensor 'batch_join:1' shape=(?,) dtype=int32>

In [76]:
image_batch = tf.identity(image_batch, 'image_batch')
image_batch = tf.identity(image_batch, 'input')
label_batch = tf.identity(label_batch, 'label_batch')

print('Number of classes in training set: %d' % nrof_classes)
print('Number of examples in training set: %d' % len(image_list))

print('Number of classes in validation set: %d' % len(val_set))
print('Number of examples in validation set: %d' % len(val_image_list))

print('Building training graph')

Number of classes in training set: 5749
Number of examples in training set: 7361
Number of classes in validation set: 5749
Number of examples in validation set: 5872
Building training graph


In [82]:
prelogits, _ = network.inference(image_batch, keep_probability, 
            phase_train=phase_train_placeholder, bottleneck_layer_size=embedding_size, 
            weight_decay=weight_decay)

In [83]:
logits = slim.fully_connected(prelogits, len(train_set), activation_fn=None, 
                weights_initializer=slim.initializers.xavier_initializer(), 
                weights_regularizer=slim.l2_regularizer(weight_decay),
                scope='Logits', reuse=False)

In [84]:
 embeddings = tf.nn.l2_normalize(prelogits, 1, 1e-10, name='embeddings')

In [8]:
[1,2] + [3]*2 

[1, 2, 3, 3]